Import packages.

In [ ]:
import time
import numpy                   as np
import matplotlib.pyplot       as plt
from sklearn.cluster           import KMeans
from matplotlib                import cm

In [ ]:
from MREC import distortion_score, MREC

Dataset.

In [ ]:
n_pts_per_clus = 2*10

In [ ]:
X = np.concatenate([np.random.normal(loc=0.,  scale=1., size=n_pts_per_clus)[:,np.newaxis],  
                    np.random.normal(loc=0.,  scale=1., size=n_pts_per_clus)[:,np.newaxis]], axis=1)
Y = np.concatenate([np.random.normal(loc=0.,  scale=1., size=n_pts_per_clus)[:,np.newaxis],  
                    np.random.normal(loc=10., scale=1., size=n_pts_per_clus)[:,np.newaxis]], axis=1)
Z = np.concatenate([np.random.normal(loc=30., scale=1., size=n_pts_per_clus)[:,np.newaxis],  
                    np.random.normal(loc=0.,  scale=1., size=n_pts_per_clus)[:,np.newaxis]], axis=1)

In [ ]:
X1 = np.concatenate([X[:int(len(X)/2),:],Y[:int(len(Y)/2),:],Z[:int(len(Z)/2),:]], axis=0)
lab1 = np.array(  [1 for _ in range(int(len(X)/2))] 
                + [2 for _ in range(int(len(Y)/2))] 
                + [3 for _ in range(int(len(Z)/2))])
X2 = np.concatenate([X[int(len(X)/2):,:],Z[int(len(Z)/2):,:],Y[int(len(Y)/2):,:]], axis=0)
lab2 = np.array(  [1 for _ in range(int(len(X)/2))] 
                + [3 for _ in range(int(len(Z)/2))] 
                + [2 for _ in range(int(len(Y)/2))])

In [ ]:
print(lab1.shape, X1.shape, lab2.shape, X2.shape)

Compute matching.

In [ ]:
bp={"mode": "WS", "metric": "euclidean", "epsilon": .1, "tol": 1e-9, "max_iter": 1000}

In [ ]:
bp={"mode": "GWS", "metric": "euclidean", "epsilon": .1, "tol": 1e-9, "max_iter": 1000}

In [ ]:
bp={"mode": "MXS", "metric": ["euclidean","euclidean"], "alpha": .5, "epsilon": .1, "tol": 1e-9, "max_iter": 1000}

In [ ]:
bp={"mode": "GWNC", "metric": "euclidean", "num_iter": 2, "method": "L-BFGS-B"}

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
bp={"mode": "GWC_sdpnal", "metric": "euclidean", "eng": eng, "use_birkhoff": False}

In [ ]:
bp={"mode": "GWC_cplex", "metric": "euclidean", "eng": eng, "maxtime": 10}

In [ ]:
start = time.time()
gamma, mapping = MREC(X1=X1, X2=X2, q_method="RandomChoice", threshold=10, n_points=30, 
                      backend_params=bp, return_gamma=True)
end = time.time()
quality = 0
for i in range(len(mapping)):
    labs, labt = lab1[i], lab2[mapping[i]]
    if labs == labt:
        quality += 1
quality /= len(mapping)
distortion = distortion_score(X1[::100], X2[::100], gamma[::100,:][:,::100], mode=3)
print(quality, distortion, end-start)

Plot matching.

In [ ]:
mapped_X = len(X2) * np.dot(gamma, X2)

In [ ]:
%matplotlib notebook

plt.figure()
labname = {1: "Group 1", 2: "Group 2", 3: "Group 3"}
clrmap = cm.get_cmap("rainbow")
ML, mL = max(lab1), min(lab1)
for i in np.unique(lab1):
    I1, I2 = np.argwhere(lab1==i)[::10,0], np.argwhere(lab2==i)[::10,0]
    plt.scatter(X1[I1,0], X1[I1,1], c=clrmap((i-mL)/(ML-mL)), marker=".", s=30, alpha=.9, label=labname[i])
    plt.scatter(X2[I2,0], X2[I2,1], c=clrmap((i-mL)/(ML-mL)), marker="+", s=30, alpha=.9)
    plt.scatter(mapped_X[I1,0], mapped_X[I1,1], c=clrmap((i-mL)/(ML-mL)), marker="*", s=30, alpha=.9)
    for j in I1:
        plt.plot([X1[j,0], mapped_X[j,0]], [X1[j,1], mapped_X[j,1]], c="black", linewidth=.1)  
plt.legend()